In [ ]:
import pandas as pd
df = pd.read_csv('data.txt', encoding='windows-1251')

In [ ]:
user_columns = ['DR_Dat', 'DR_Tim', 'DR_NChk', 'DR_NDoc', 
                'DR_Apt', 'DR_NDrugs', 'DR_Kol', 'DR_CZak', 'DR_CRoz', 
                'DR_SDisc', 'DR_TPay', 'DR_CDrugs',  
                'DR_Suppl','DR_CDisc', 'DR_BCDisc', 'DR_TabEmpl',
                'DR_VZak', 'DR_Pos']

df = df[user_columns]

In [ ]:
df.columns = ['date', 'time', 'n_chk', 'n_doc', 'apt', 'name_drugs',
       'amnt', 'purch_price', 'retail_price', 'disc', 'pay_type', 'drug_id',
       'supplier', 'disc_id', 'disc_barcode', 'employee', 'vzak', 'pos']

In [ ]:
df['disc_barcode'] = df['disc_barcode'].astype('str').replace(r'\.0', '', regex=True)
df['date']= pd.to_datetime(df['date']).dt.strftime('%d.%m.%Y')
df['vzak'] = df['vzak'].astype('str').replace('1', 'offline').replace('2', 'online')
df = df.fillna(0)

In [ ]:
df.head(10)

In [ ]:
# this function shows structure of the dataframe
df.info()

In [ ]:
# i can use describe method to get most of the descriptive parameters of the dataframe
df.describe()
# this method only shows non-objective type columns - only int and float type columns will be shown

In [ ]:
# to work with categorical data i need to use construction like this
df['pay_type'].value_counts()

In [ ]:
# i also can call column by .
df.pay_type.value_counts()

In [ ]:
df.apt.value_counts().sort_values()

In [ ]:
# % from all
df.apt.value_counts(normalize=True)

In [ ]:
# using a histogram to estimate a distribution
df.pos.value_counts() # this does almost what i need, but its not fancy
df.pos.hist() # will build histogram
df.pos.hist(bins=24) # i can set custom bins

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# pandas couldnt convert it to float, stated that this is str
df['time'] = pd.to_datetime(df['time']).dt.strftime('%H:%M:%S')

In [ ]:
# pandas couldnt convert it to float, stated that this is str. I clearly changed type
df['name_drugs'] = df.name_drugs.astype('str')

In [ ]:
# pandas couldnt convert it to float, stated that this is str. I clearly changed type, but it didnt help
df['supplier'] = df.supplier.astype('str').replace('\"', '', regex=True).replace(' ', '_', regex=True)

In [ ]:
# checking what it sees as number columns
df.select_dtypes(include='number').columns 

In [ ]:
# one of ways to find correlation is to build heatmap
# i need to give correlation matrix of my dataframe
plt.figure(figsize=(16, 6)) # this way i can customize the look of heatmap
sns.heatmap(df.corr(numeric_only=True), vmax=1, vmin=-1, annot=True) # specified that it should use only numeric columns
# also added vmax and vmin to clearly see positive and negative correlation

In [ ]:
# i created new column so i can evaluate the distribution of sold items by hour
df['hour'] = pd.to_datetime(df.time).dt.hour

In [ ]:
# in my data file i need to count column 'amnt' by each hour and see distribution
ds = df.groupby(['hour'])['amnt'].agg('sum')
ds = df.groupby(['hour']).agg({'amnt': 'sum'}) # does the same
# logic is similar to sql -- im summing column 'amnt' and grouping it by column 'hour'
# im assigning it to a new variable since i need to male boxplot

In [ ]:
plt.figure(figsize=(3, 6))
sns.boxplot(ds)

In [ ]:
# here is more complex example of grouping
# i can set rules of agg in dictionary - column name would be a key and value is what i need to do (count, sum, max, etc)
a = df.groupby(['date', 'n_chk'])[['amnt', 'retail_price', 'purch_price']].agg({
    'amnt': 'sum', 
    'retail_price': ['sum', 'max'],
    'purch_price': 'sum'
})

In [ ]:
a.head()
# here the grouping column became index, not a separate columns

In [ ]:
# with this function i can make them separate columns
a.reset_index()

In [ ]:
# here is an example of counting revenue for every day by every employee
# agg functin cant accept math operations inside, so one way around is to make a new column
rev_1 = df.groupby(['date', 'employee'])[['retail_price', 'purch_price']].agg('sum')
rev_1['revenue'] = rev_1['retail_price'] - rev_1['purch_price']
rev_1['revenue'] = rev_1['revenue'].round(2) # here i rounded values in revenue column
rev_1 = rev_1.reset_index() # here i reassigned rev datarfame


In [ ]:
# here i sorted it by revenue in descending order (ascending by default)
rev_1.sort_values(by='revenue', ascending=False)

In [ ]:
# second way is using 'apply' function after grouping
# it will apply everything i put in it to all columns - in this case its 'retail_price' and 'purch_price'
rev_2 = df.groupby(['date', 'employee'])[['retail_price', 'purch_price', 'amnt']].apply(lambda x: sum(x['amnt'] * (x['retail_price'] - x['purch_price'])))
# in this case x in lambda is a row in a column
rev_2 = rev_2.reset_index()
# this way is better becouse it is easier to add new variables such as amnt in this example

In [ ]:
# changed names for columns
rev_2.columns = ['date', 'employee', 'revenue']
rev_2['revenue'] = rev_2['revenue'].round(2)

In [ ]:
# sorted by date and revenue in descending order
rev_2.sort_values(['date','revenue'], ascending=False)

In [ ]:
# this is how i can round by hundreds
df['r_retail_price'] = df.retail_price.apply(lambda x: round(x, -2))

In [ ]:
diag = df.groupby(['date', 'r_retail_price'])['amnt'].agg('sum').reset_index()

In [ ]:
# to make scatterplot i can use seaborn
sns.scatterplot(data=diag, x='r_retail_price', y='amnt', hue='date')

In [ ]:
sns.jointplot(data=diag, x='r_retail_price', y='amnt', hue='date')

In [ ]:
# to make this data to look more clean and readable i can make pivot tables 
df.groupby(['date', 'apt', 'employee', 'pay_type'])[['amnt']].agg('sum')

In [ ]:
pt = df.groupby(['date', 'apt', 'employee', 'pay_type'])[['amnt']].agg('sum').reset_index()

In [ ]:
pt.pivot_table(values='amnt', index=['date', 'apt', 'employee'], columns='pay_type')

In [ ]:
# i can also do pivot table from initial dataframe, without groupby and agg steps
df.pivot_table(values='amnt', index=['date', 'apt', 'employee'], columns='pay_type', aggfunc='sum')
# i just need to add one extra argument 'aggfunc' - it will tel what type of agg i need for values

In [ ]:
# i also can unpivot tables
pvt = df.pivot_table(values='amnt', index=['date', 'apt', 'employee'], columns='pay_type', aggfunc='sum').reset_index()

In [ ]:
#id_vars is the same as index above, what stays unchanged
# value_vars is what i need to unpivot - change from columns to one column 'pay_type'
pd.melt(pvt, id_vars=['date', 'apt', 'employee'], value_vars=[15, 18], var_name='changed_back_pay_type')
# last argument is for changing name for a new column

In [ ]:
# both these functions - pivot_table and melt are often used in tasks like changing long table to wide and vice versa

In [ ]:
# if i need to find something specific like 'positions which sold more then 5 times' or 'where price is higher then ...' i can use this
# like in sql 'where something something'
df.iloc[5,5] # this command will return 5th row in 5th column
df.iloc[:5, 5] # this will return all rows till 5th (not including it) in 5th column
df.iloc[:5, :5] # all rows and columns before 5
df.iloc[1:5, 2:4] # this will give from 1 to 5 rows in 2 to 4 columns (not including last one)

In [ ]:
# iloc only works with indexes, if i want to search by column names i can use loc
df.loc[2:5, ['date', 'apt']] # i need rows from 2 to 5 in columns 'date' and 'apt'

In [ ]:
# so to look by some specific parameter line 'where something something' i can use this
df['amnt'] > 5 # this will return massive with True\False, so i need to index that
df[df['amnt']>5] # this will return all rows where value in column 'amnt' is more then 5

In [ ]:
# i can use 'and - &' or 'or - |' 
# every conditions must be in ()
df[(df['amnt'] > 5) | (df['retail_price'] > 2000)]
df[(df['amnt'] > 5) & (df['retail_price'] > 30)]

In [ ]:
# if i want to see only rows where 'pay_type' is not 15 and not 20
df[df.loc[: , 'pay_type'].isin([15, 20])] # this function check if values in column are in massive and returns bool df

In [ ]:
df[~df.loc[: , 'pay_type'].isin([15, 20])] #by adding ~ in front of the condition i can reverse it

In [ ]:
# so this filtering is a good practice with building scatterplot
diag = df.groupby(['date', 'r_retail_price'])['amnt'].agg('sum').reset_index()
diag = diag[diag.r_retail_price < 4000]

In [ ]:
sns.scatterplot(data=diag, x='r_retail_price', y='amnt', hue='date' )
# i discarded spikes from scatterplot and looks much better now